In [15]:
# Installing packages
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install keras
!pip install tensorflow
!pip install scikeras

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [2]:
# Load the data
train = pd.read_csv('data\Final_data_after_feature_engineering\Rwanda_CO2_emission_dataset_train.csv')
test = pd.read_csv('data\Final_data_after_feature_engineering\Rwanda_CO2_emission_dataset_test.csv')

In [3]:
# Select the features and target variable for training
X_train = train.drop('emission', axis=1)
y_train = train['emission']

# Select the features for testing
X_test_imputed = test

In [4]:
# Split the training data into a training and validation set
X_train_imputed, X_val_imputed, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [5]:
# Select all columns after 'week_no' from each DataFrame (selecting only relevent features)
X_train_selected = X_train_imputed.iloc[:, 1:]
X_val_selected = X_val_imputed.iloc[:, 1:]
X_test_selected = X_test_imputed.iloc[:, 1:]

In [6]:
# Normalize the data using min-max normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [7]:
# Reshape the data for LSTM
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_val_reshaped = np.reshape(X_val_scaled, (X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [8]:
def create_model(lr=0.01):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(50))
    model.add(LeakyReLU(alpha=0.01))  # Leaky ReLU activation
    model.add(Dense(1, activation='linear'))  # Linear activation for output
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))
    return model


In [9]:
# Wrap the Keras model in a scikit-learn compatible estimator
model = KerasRegressor(build_fn=create_model)

In [ ]:
# Perform hyperparameter tuning using GridSearchCV for LSTM model
param_grid_lstm = {'batch_size': [124, 248], 'epochs': [50]}
grid_search_lstm = GridSearchCV(model, param_grid=param_grid_lstm,cv=3)
grid_search_lstm.fit(X_train_reshaped,y_train)

C:\Users\singi\anaconda3\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/50
340/340 [==============================] - 14s 13ms/step - loss: 33857.9961
Epoch 2/50
340/340 [==============================] - 4s 12ms/step - loss: 33600.2461
Epoch 3/50
340/340 [==============================] - 4s 12ms/step - loss: 33617.8555
Epoch 4/50
340/340 [==============================] - 4s 12ms/step - loss: 33624.2695
Epoch 5/50
340/340 [==============================] - 4s 12ms/step - loss: 33591.3633
Epoch 6/50
340/340 [==============================] - 4s 12ms/step - loss: 33595.7305
Epoch 7/50
340/340 [==============================] - 4s 12ms/step - loss: 33585.9922
Epoch 8/50
340/340 [==============================] - 4s 12ms/step - loss: 33593.3438
Epoch 9/50
152/340 [============>.................] - ETA: 2s - loss: 36664.3984

In [ ]:
# Train the LSTM model using the best hyperparameters found by GridSearchCV
best_params_lstm=grid_search_lstm.best_params_
model.fit(X_train_reshaped,y_train,epochs=best_params_lstm['epochs'],batch_size=best_params_lstm['batch_size'],validation_data=(X_val_reshaped,y_val))

In [ ]:
# Make predictions on the test data using the LSTM model
y_pred_lstm=model.predict(X_test_reshaped)

In [ ]:
# Make predictions on the test data using a Random Forest Regressor with hyperparameter tuning
rf_model=RandomForestRegressor()
param_grid_rf={'n_estimators':[10,50],'max_depth':[None,10],'min_samples_split':[2,5]}
grid_search_rf=GridSearchCV(rf_model,param_grid=param_grid_rf,cv=3)
grid_search_rf.fit(X_train_scaled,y_train)
best_params_rf=grid_search_rf.best_params_
rf_model.set_params(**best_params_rf)
rf_model.fit(X_train_scaled,y_train)
y_pred_rf=rf_model.predict(X_test_scaled)

In [ ]:
# Make predictions on the test data using a Gradient Boosting Regressor with hyperparameter tuning
gb_model=GradientBoostingRegressor()
param_grid_gb={'n_estimators':[10,50],'max_depth':[None,10],'min_samples_split':[2,5]}
grid_search_gb=GridSearchCV(gb_model,param_grid=param_grid_gb,cv=3)
grid_search_gb.fit(X_train_scaled,y_train)
best_params_gb=grid_search_gb.best_params_
gb_model.set_params(**best_params_gb)
gb_model.fit(X_train_scaled,y_train)
y_pred_gb=gb_model.predict(X_test_scaled)

In [ ]:
# Combine the predictions from the LSTM model and the two ensemble models using a weighted average
y_pred_combined=(y_pred_lstm.flatten()+y_pred_rf+y_pred_gb)/3

In [ ]:
# Save the predictions to a CSV file
submission=pd.DataFrame({'ID_LAT_LON_YEAR_WEEK':test['ID_LAT_LON_YEAR_WEEK'],'emission':y_pred_combined})
submission.to_csv('submission_2.csv',index=False)

In [ ]:
submission.shape